Изобразить эпсилон - спектр, эллиптический спектральный портрет с полуосями a=||A||,b=t и соответствующую клеточно диагональную форму матрицы А при с=1, е=10^-3

In [91]:
import numpy as np
import numpy.linalg as la
from matplotlib import pyplot
from mpseudo import pseudo
import math
#import PyGnuplot as gp
# 
c=1
eps=pow(10,-3)



# Создаем матрицу из задания

A=np.zeros((9,9))
for i in [1,4,7]:
    A[i-1][i]=c
    A[i][i-1]=-c
    A[i][i+1]=c
    A[i+1][i]=-c
A[3,2]=eps
A[6,5]=eps
A[2,3]=-eps
A[5,6]=-eps

# Расчет нормы и собственных значений
a=la.norm(A)
eigen_values=la.eig(A)[0]
eigen_values.imag.sort()

# Расчитываем эпсилон-спектр 
def f(x,y,A):
    ones=np.zeros((9,9))
    for i in range(0,len(ones)):
        ones[i][i]=1
    res=A-ones.dot(x+1j*y)
    U,S,V=la.svd(res)
    sing_min=S.min()
    return 1/sing_min

# Получаем данные для построения эпсилон спектра
# Делаем данные
X = np.arange(-4, 4, 0.1)
Y = np.arange(-4, 4, 0.1)
X, Y = np.meshgrid(X, Y)
Z=[]
for i in range(0,len(X)):
    row=[]
    for j in range(0,len(X[0])):
        row.append(f(X[i,j],Y[i,j],A))
    Z.append(row)
Z=np.asarray(Z)

# Пишем в файл данные для графика эпсилон спектра
# splot "C:\\Users\\Pudov\\Python\\DGLA3\\eps.txt" using 1:2:3 with lines
file = open('eps.txt', 'w')
file.write("# X Y Z"+'\n')
for i in range(0,len(X)):
    for j in range(0,len(X[0])):
        if (Z[i][j]<=50):
            file.write(str(X[i][j])+' '+str(Y[i][j])+' '+str(Z[i][j])+'\n')
file.close()

# Вывод мнимых частей собственных значений в виде таблицы
from IPython.display import HTML, display
import tabulate
table = np.array([eigen_values.imag[:3],eigen_values.imag[3:6],eigen_values.imag[6:9]])
display(HTML(tabulate.tabulate(table, tablefmt='html',floatfmt=(".15f",".15f",".15f"))))

# Вывод действительных частей в виде таблицы
from IPython.display import HTML, display
import tabulate
table = np.array([eigen_values.real[:3],eigen_values.real[3:6],eigen_values.real[6:9]])
display(HTML(tabulate.tabulate(table, tablefmt='html')))




-1.414567248357250,-1.414213739149824,-1.413860141553966
-0.000707106692798,0.000000000000000,0.000707106692798
1.413860141553966,1.414213739149824,1.414567248357250


2.77556e-17,2.77556e-17,2.77556e-17
2.77556e-17,5.55112e-17,5.55112e-17
2.48666e-18,-1.83501e-17,-1.83501e-17


In [92]:
a

3.464102192487976